# Generate a video for the course of "Application of Deep Learning"
Code was generated by ChatGPT 4o (Aug. 6, 2024)

In [1]:
!pip install gtts

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 8.2.1
    Uninstalling click-8.2.1:
      Successfully uninstalled click-8.2.1


In [8]:
import moviepy.editor as mp
import gtts
from PIL import Image, ImageDraw, ImageFont, ImageOps

# 1) Images + (min) durations in seconds
images = [
    ("image1.jpg", 6),
    ("image2.jpg", 4),
    ("image3.jpg", 7)
]

# 2) Cooking-themed voiceovers (one per image)
voiceover_texts = [
    "Aprons on! Fresh herbs, garlic, and bright veggies hit the board—listen to that crisp chop.",
    "Into the pot go tomatoes, olive oil, and basil. We season, taste, and let the sauce gently simmer.",
    "Time to plate: juicy mains with a bright salad. A squeeze of lemon and dinner is served."
]

# 3) Add multi-line caption to an image and save a 1280x720 version
#    y_offset now acts as the *bottom margin* in pixels (default 40).
def add_text_to_image_multiline_black(image_path, text, output_path, font_size=40, max_width=50, y_offset=40):
    img = Image.open(image_path).convert("RGB")
    img = ImageOps.pad(img, (1280, 720), color=(0, 0, 0))  # letterbox to 16:9

    draw = ImageDraw.Draw(img)
    # Font fallback so it works in Colab even if LiberationSans isn't present
    try:
        font = ImageFont.truetype("LiberationSans-Bold.ttf", font_size)
    except Exception:
        try:
            font = ImageFont.truetype("DejaVuSans.ttf", font_size)
        except Exception:
            font = ImageFont.load_default()

    # Simple word-wrap by character count (kept like your original)
    words = text.split()
    lines = []
    while words:
        line = ""
        while words and len(line) + len(words[0]) + 1 <= max_width:
            line += words.pop(0) + " "
        lines.append(line.strip())

    # Compute total text block height
    heights = []
    widths = []
    for ln in lines:
        bbox = draw.textbbox((0, 0), ln, font=font)
        w = bbox[2] - bbox[0]
        h = bbox[3] - bbox[1]
        widths.append(w)
        heights.append(h)
    total_text_height = sum(heights) + (len(lines) - 1) * (heights[0] // 1)  # spacing similar to your *2

    width, height = img.size
    # Anchor near bottom: bottom margin = y_offset
    y = height - y_offset - total_text_height

    # Draw each line centered, with a black rectangle behind for readability
    for idx, line in enumerate(lines):
        bbox = draw.textbbox((0, 0), line, font=font)
        text_w = bbox[2] - bbox[0]
        text_h = bbox[3] - bbox[1]
        x = (width - text_w) / 2
        draw.rectangle([(x - 10, y - 10), (x + text_w + 10), (y + text_h + 10)], fill="black")
        draw.text((x, y), line, font=font, fill="white")
        # spacing: roughly one line height extra (matches your previous visual)
        y += text_h * 2

    img.save(output_path)

# 4) Create captioned images
captioned_images = []
for i, (image_path, duration) in enumerate(images):
    output_path = f"captioned_image_{i}.png"
    add_text_to_image_multiline_black(image_path, voiceover_texts[i], output_path)
    captioned_images.append((output_path, duration))

# 5) Generate voiceovers with gTTS
voiceover_clips = []
for i, text in enumerate(voiceover_texts):
    mp3_path = f"voiceover_{i}.mp3"
    tts = gtts.gTTS(text, lang='en')
    tts.save(mp3_path)
    voiceover_clips.append(mp.AudioFileClip(mp3_path))

# 6) Build per-image clips (duration = max(specified, audio length + small pad))
image_clips = []
for i, (image_path, duration) in enumerate(captioned_images):
    audio = voiceover_clips[i]
    dur = max(duration, audio.duration + 0.25)  # tiny pad so VO doesn't cut off
    clip = mp.ImageClip(image_path).set_duration(dur).set_audio(audio)
    image_clips.append(clip)

# 7) Concatenate and export video
final_clip = mp.concatenate_videoclips(image_clips, method="compose")
output_file_path = "class_exercise.mp4"
final_clip.write_videofile(output_file_path, fps=24, codec="libx264", audio_codec="aac", bitrate="2000k")

output_file_path


Moviepy - Building video class_exercise.mp4.
MoviePy - Writing audio in class_exerciseTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video class_exercise.mp4



Moviepy - Done !
Moviepy - video ready class_exercise.mp4


'class_exercise.mp4'